# Chord inversions

The idea of a chord inversion can be interpreted in multiple ways.

## Inverting pitch sets

Inverting a pitch set means taking the lowest pitch and moving it to the top,
or taking the highest pitch and moving it to the bottom.

For example, you can have a pitch set `[0,4,7]` and invert it by moving the lowest pitch, `0`,
to the top of the chord, yielding `[4,7,12]`.

You can also take the highest pitch, `7`, and move it to the bottom, yielding `[-5,0,4]`.

### Parameterizing the algorithm

There are two obvious parameters here so far:

- The pitch set
- The amount it will be inverted

If the inversion amount is positive, then we are moving the lowest pitches to the top.

Conversely, if the amount is negative, then we are moving the highest pitches to the bottom.

There's just one parameter missing here: when we "move the lowest pitch to the top" or "move the highest pitch to the bottom",
we are adding / subtracting a certain amount to that pitch. How do we determine how much to add to/subtract from that pitch?

For example, when we invert `[0,4,7]` by -1 and get `[-5,0,4]`, we are subtracting 12 from the highest pitch, `7`.
**How did we get that number?**

Well, it's because when we transpose the pitch, we want to preserve the pitch class.

That is to say, if the pitch we're transposing has a pitch class of 7, then we transposed pitch should also have a pitch class of 7.

So, if we're inverting the chord with a negative amount, then we want to transpose the highest pitch downward,
giving us the greatest pitch that's less than all the other pitches and has the same pitch class as that highest pitch.

Every pitch class has a modulus, and this is the final parameter: **the modulus!**

Putting this all together, we have our algorithm for inverting a pitch set.

In [ ]:
from harmonica.pitch import PitchSet


def invert_pitch_set(pitch_set: PitchSet, amount: int, modulus: int) -> PitchSet:
    new_pitch_set = pitch_set.pitches

    if amount > 0:  # Moving lowest pitch to top of pitch set
        for _ in range(abs(amount)):
            lowest_pitch = new_pitch_set[0]
            highest_pitch = new_pitch_set[-1]
            while lowest_pitch <= highest_pitch:
                lowest_pitch += modulus  # Add modulus to the lowest pitch
            new_pitch_set = new_pitch_set[1:] + [lowest_pitch]
    elif amount < 0:
        for _ in range(abs(amount)):
            lowest_pitch = new_pitch_set[0]
            highest_pitch = new_pitch_set[-1]
            while highest_pitch >= lowest_pitch:
                highest_pitch -= modulus
            new_pitch_set = [highest_pitch] + new_pitch_set[:-1]

    return PitchSet(new_pitch_set)


pset = PitchSet([0, 7, 14])
print(invert_pitch_set(pset, 2, 12))  # Should print [14, 24, 31]
print(invert_pitch_set(pset, -3, 12))  # Should print [-24, -17, -10]

PitchSet(pitches=[14, 24, 31])
PitchSet(pitches=[-24, -17, -10])


### Inverting in place

The previous kind of inversion is great, but there is another way to invert chords.

Say you have the pitch set `[0,4,7]`. What if you want to invert this to first inversion,
like `[4,7,12]`, but you want to fix the lower pitch - i.e, keep it equal to 0?

Essentially, all you need to do is perform the previous inversion algorithm, and then transpose
the resulting pitch set such that its lowest pitch is equal to the lowest pitch in the original pitch set.

So, let's do this to `[0,3,7,10]` with an amount of 2 and a modulus of 12. First, we invert upwards twice:

```
0 3 7 10
  3 7 10 12
    7 10 12 15
```

Then, we transpose it so the bass pitch remains fixed at 0, i.e. by -7:

```
7 10 12 15
  v -7 v
0  3  5  8
```

And there you have it: the result is `[0,3,5,8]`.

_In fact_, you mustn't limit yourself to the bass pitch - you can fix any of the pitches.
For example, say you want to invert `[0,3,7,10]` with an amount of 2 and a modulus of 12,
fixed around the pitch at index 2, which is `7`.

First, you invert upward twice again:

```
0 3 7 10
  3 7 10 12
    7 10 12 15
```

Now, you transpose `[7,10,12,15]` such that the pitch at index 2, `12`, is equal to the pitch
at index 2 of the original pitch set, which was `7`. For this, we transpose by -5:

```
7 10 12 15
  v -5 v
2  5  7 10
```

And this gives us our result of `[2,5,7,10]`.

In [ ]:
from harmonica.pitch import PitchSet


def invert_pset_fixed_pitch(
    pitch_set: PitchSet, amount: int, modulus: int, fixed_index: int
) -> PitchSet:
    inverted_pset = invert_pitch_set(pitch_set, amount, modulus)
    transpose = pitch_set[fixed_index] - inverted_pset[fixed_index]

    return inverted_pset + transpose


pset = PitchSet([0, 3, 7, 10])
print(invert_pset_fixed_pitch(pset, 2, 12, 2))  # Should print [2,5,7,10]

PitchSet(pitches=[2, 5, 7, 10])


### Fun example

Let's construct a sequence of pitch sets, starting with `[0,3,7,10]` and then going through all inversions fixed at pitch 0,
then all inversions fixed at pitch 3, pitch 7, and then pitch 10.

In [ ]:
from fractions import Fraction
from harmonica.pitch import PitchSetSeq

pset = PitchSet([0, 3, 7, 10])

inversions = []

for index in range(len(pset)):
    for amount in range(len(pset)):
        inversions.append(
            invert_pset_fixed_pitch(pset, amount, modulus=12, fixed_index=index)
        )

# Uncomment to preview
# PitchSetSeq(inversions).preview(bass=60, duration=Fraction(4), program=89)